# Characters Data Acquisition

This notebook fetches a JSON file with data of Genshin Impacts' Characters. It uses a free fan-made API: https://github.com/genshindev/api

And exports it in a tabular format suitable for consumption by Pandas.

In [ ]:
import json
import requests

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from pathlib import Path

from typing import *
from numpy.typing import NDArray

In [ ]:
DATA_DIR = Path('../data')

# Data Fetching

In [ ]:
print('#--------------------------#')
print('| Fetching Characters Data |')
print('#--------------------------#')
print('')

response = requests.get('https://genshin.jmp.blue/characters/all')

print('Status Code:', response.status_code)
assert response.status_code == 200

print(f'Content Type:', response.headers.get('Content-Type'))
assert 'application/json' in response.headers.get('Content-Type', '')

print(f'Response Size: {len(response.content) / 1024:.1f} kB')
data = response.json()

print('')
print('#-----------------------------------#')
print('| Fetched Characters Successfully ✓ |')
print('#-----------------------------------#')
print('')

print('Total Characters:', len(data))

# Subtables Creation

In [ ]:
data_skills = [] # skills_talents
data_upgrades = [] # skills_talents_upgrades
data_passive = [] # passive_talents
data_constillations = [] # constillations
data_outfits = [] # outfits

for character in data:
    for skill in character['skillTalents']:
        data_skills.append({'character': character['id'], **skill})
        for upgrade in skill.get('upgrades', []):
            data_upgrades.append({'character': character['id'], 'type': skill.get('type', 'RIGHT_CLICK'), **upgrade})
    
    for skill in character['passiveTalents']:
        data_passive.append({'character': character['id'], **skill})

    for constillation in character['constellations']:
        data_constillations.append({'character': character['id'], **constillation})

    for outfit in character.get('outfits', []):
        data_outfits.append({'character': character['id'], **outfit})

# DataFrames Creation

## Characters

In [ ]:
df_characters = pd.DataFrame(data)
df_characters.set_index('id', inplace=True)
df_characters.drop(columns=['skillTalents', 'passiveTalents', 'constellations', 'outfits'], inplace=True)
df_characters['rarity'] = df_characters['rarity'].astype(np.int8)
df_characters['release'] = pd.to_datetime(df_characters['release'], format='%Y-%m-%d', utc=True)
for name in ['vision', 'weapon', 'gender', 'nation', 'affiliation', 'vision_key', 'weapon_type']:
    df_characters[name] = df_characters[name].astype('category')

## Skills Talents

In [ ]:
df_skills = pd.DataFrame(data_skills)
df_skills.drop(columns='upgrades', inplace=True)
df_skills['unlock'] = df_skills['unlock'].str.title()
df_skills['type'].fillna('RIGHT_CLICK')
for name in ['character', 'unlock', 'type']:
    df_skills[name] = df_skills[name].astype('category')
df_skills.set_index(['character', 'name'], inplace=True)

## Skills Talents Upgrades

In [ ]:
df_upgrades = pd.DataFrame(data_upgrades)
for name in ['character', 'type', 'name']:
    df_upgrades[name] = df_upgrades[name].astype('category')
df_upgrades.set_index(['character', 'type'], inplace=True)

## Passive Talents

In [ ]:
df_passive = pd.DataFrame(data_passive)
for name in ['character', 'unlock']:
    df_passive[name] = df_passive[name].astype('category')
df_passive['level'] = df_passive['level'].dropna().astype(np.int8).astype('category')
df_passive.set_index(['character', 'name'], inplace=True)

## Constillations

In [ ]:
df_constillations = pd.DataFrame(data_constillations)
df_constillations['level'] = df_constillations['level'].astype(np.int8)
for name in ['character', 'unlock']:
    df_constillations[name] = df_constillations[name].astype('category')
df_constillations.set_index(['character', 'name'], inplace=True)

## Outfits

In [ ]:
df_outfits = pd.DataFrame(data_outfits)
df_outfits['rarity'] = df_outfits['rarity'].astype(np.int8)
for name in ['character', 'type']:
    df_outfits[name] = df_outfits[name].astype('category')
df_outfits.set_index(['character', 'name'], inplace=True)

# DataFrames Preview

## Characters

In [ ]:
df_characters.head(2)

## Skills Talents

In [ ]:
df_skills.head(2)

## Skills Talents Upgrades

In [ ]:
df_upgrades.head(2)

## Passive Talents

In [ ]:
df_passive.head(2)

## Constillations

In [ ]:
df_constillations.head(2)

## Outfits

In [ ]:
df_outfits.head(2)

# DataFrames Export

In [ ]:
for name in ['characters', 'skills', 'upgrades', 'passive', 'constillations', 'outfits']:
    df = globals()[f'df_{name}']
    path_csv = DATA_DIR.joinpath(f'genshin-{name}.csv')
    path_parquet = DATA_DIR.joinpath(f'genshin-{name}.parquet')
    
    df.to_csv(path_csv)
    print('Exported', path_csv)
    df.to_parquet(path_parquet)
    print('Exported', path_parquet)